Given a function $f$ with suitable integrability, the Hilbert transform of $f$ is defined as
$$
H(f)(x) \coloneqq \frac{1}{\pi} \text{p.v.}\int_{-\infty}^\infty \frac{f(y)}{x-y} \ dy.
$$
We employ interval arithmetic to compute the Hilbert transform of a function $f$ using Gaussian-Legendre quadrature with rigorous error bounds. The idea of Gaussian-Legendre quadrature is to approximate an integral with a sum. That is,
$$
\int_{-1}^1 g(x) \ dx \approx \sum_{i=1}^N w_i g(x_i)
$$
for some nodes $x_i$ and weights $w_i$ to be determined. Given this integral over $[-1,1]$, we can then integrate $g$ over any interval $[a,b]$ using a change of coordinates to get
$$
\int_a^b f(x) \ dx \approx \frac{b-a}{2}\sum_{i=1}^N w_ig(\frac{b-a}{2}x_i + \frac{a+b}{2}).
$$
The idea in identifying the nodes and weights is to use interpolating orthogonal polynomials. In our case, the Legendre polynomials are used, and the $N$ nodes are the $N$ roots of the $N$-th Legendre polynomial. The error of our approximation is given by
$$
\mathcal{E} \leq \frac{(b-a)^{2N+1}(N!)^4}{(2N+1)[(2N)!]^3}|f^{(2N)}(\xi)|
$$
for some $\xi \in (a,b)$. 

We note that $f$ must have $2N$ continuous derivatives for this to make sense. The Hilbert transform, however, has a singularity, so we must integrate by parts until we have something that makes sense.

For simplicity, let us take $f(x) = xe^{-x^2}$ and compute the Hilbert transform $H(f)$ of the Gaussian $f$ using a $4$-point Gaussian quadrature. In this case, we want something $C^8$ in the integrand, so we must use the rapid decay of $f$ to integrate by parts to define the Hilbert transform differently. For any $g \in \mathcal{S}(\mathbb{R})$ (the Schwartz class), one application of integration by parts gives
$$
H(g)(x) \coloneqq -\frac{1}{\pi}\int_{-\infty}^\infty \log(|x-y|)g'(y) \ dy.
$$
This integrand is not continuous, but integrating $\log(|x-y|)$ gives something continuous, so we iteratively apply integration by parts until we get something suitably regular.

Choose points $y_0,\ldots, y_N \in \mathbb{R}$. Then we can write
$$
H(g)(x) = \sum_{k=1}^N \int_{y_{k-1}}^{y_k} K(x-y)g^{(m)}(y) \ dy + \int_{-\infty}^{y_0} K(x-y)g^{(m)}(y) \ dy + \int_{y_N}^\infty K(x-y)g^{(m)}(y) \ dy.
$$
Dealing with the integrals at infinity separately, we can focus on computing the sum
$$
\sum_{k=1}^N \int_{y_{k-1}}^{y_k} K(x-y)g^{(m)}(y) \ dy.
$$
We apply symbolic integration to get the functions $K$ and $g^{(m)}$ from integration by parts. I use Maple since I didn't have time to figure out Sage (it was giving something weird when integrating symbolically).


In [2]:
import sympy as sp

# Set interval arithmetic precision
prec = 200
RBF = RealBallField(prec)
RIF = RealIntervalField(prec)

def nodes():
    x0 = -RBF(sqrt(3/7 + 2/7 * sqrt(6/5)))
    x1 = -RBF(sqrt(3/7 - 2/7 * sqrt(6/5)))
    return [x0, x1, -x1, -x0]

def weights():
    w0 = RBF((18 + sqrt(30))/36)
    w1 = RBF((18 - sqrt(30))/36)
    return [w1, w0, w0, w1]

def rigorous_gauss_quad(f, a, b, node_list, weight_list):
    n = 4 
    ans = 0
    for k in range(n):
        shifted_node = node_list[k]*(b-a)/2 + (a+b)/2
        ans += weight_list[k] * f(x = shifted_node)

    return ans * (b-a)/2

def rigorous_error(f, a, b):
    n = int(4)
    m = int(2*n)
    I = RIF(a,b)
    sup = 0

    # Find a bound on of |f| over I.
    # Note that I is small, so even if bound is rough it shouldn't be that bad?
    m1 = abs(RIF(f(x=I)).upper())
    m2 = abs(RIF(f(x=I)).lower())

    sup = max(m1,m2)

    err = ((b-a)**(m+1)*(factorial(n))**4) / ((m+1)*(factorial(m))**3) * RBF(sup)

    # multiply by [-1,1]  +/- err

    return err

# Define function found symbolically in Maple (function to be integrated)
def func(x):
    return exp(x)

# Also define its 8-th derivative
def d_func(x):
    return exp(x)

x = var('x')
a = RBF(0)
b = RBF(0.01)

gq = rigorous_gauss_quad(func(x), a, b, nodes(), weights())
err = rigorous_error(d_func(x), a, b)

print(gq)
print(err) # The actual error is not this, just an upper bound for the error.
print(gq + err) 


[0.010050167084168057752424384748270738174383934746480594001248 +/- 4.12e-61]
[5.6804642744228413763445684180029526974130288813716307245997e-28 +/- 5.20e-87]
[0.010050167084168057752424385316317165616668072380937435801543 +/- 1.59e-61]
